In [1]:
!nvidia-smi

Wed Dec 31 05:20:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P0             31W /   70W |    1617MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os

# Kaggle data path
data_path = "/kaggle/input/mitre-datset"

if os.path.exists(data_path):
    files = os.listdir(data_path)
    print("📂 Files in data folder:")
    for f in files:
        size = os.path.getsize(os.path.join(data_path, f)) / (1024**2)
        print(f"   {f}: {size:.1f} MB")
    
    # Verify JSONL
    import json
    for filename in ['train.jsonl', 'val.jsonl', 'test.jsonl']:
        filepath = os.path.join(data_path, filename)
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as f:
                count = sum(1 for _ in f)
            print(f"✅ {filename}: {count:,} examples")
        else:
            print(f"❌ {filename}: NOT FOUND")
else:
    print("❌ Dataset not found!")
    print("📌 Upload data: Click 'Add Data' > 'Upload' > 'New Dataset'")
    print("   Name it: mitre-dataset")
    print("   Upload: train.jsonl, val.jsonl, test.jsonl")

📂 Files in data folder:
   val.jsonl: 48.9 MB
   test.jsonl: 57.1 MB
   train.jsonl: 395.8 MB
✅ train.jsonl: 31,409 examples
✅ val.jsonl: 3,717 examples
✅ test.jsonl: 4,352 examples


In [3]:
# Install required packages (no bitsandbytes for FP16 training)
print("📦 Installing dependencies...")
!pip install -q transformers datasets accelerate peft
print("✅ Dependencies installed!")

📦 Installing dependencies...
✅ Dependencies installed!


In [4]:
# Cell 4: Configuration for Kaggle
import torch

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"  # 1.5B fits well in GPU

# Kaggle paths (CHANGED from Colab)
DATA_PATH = "/kaggle/input/mitre-datset"
OUTPUT_DIR = "/kaggle/working/checkpoints"
FINAL_MODEL_DIR = "/kaggle/working/fine_tuned_model"

TRAIN_FILE = f"{DATA_PATH}/train.jsonl"
VAL_FILE = f"{DATA_PATH}/val.jsonl"

# Training settings
MAX_LENGTH = 512
BATCH_SIZE = 1
GRAD_ACCUM_STEPS = 8
NUM_EPOCHS = 2           # 2 epochs for good fine-tuning
LEARNING_RATE = 2e-4

print("✅ Configuration loaded (Kaggle)")
print(f"   Model: {MODEL_NAME}")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Estimated time: ~3-4 hours")

✅ Configuration loaded (Kaggle)
   Model: Qwen/Qwen2.5-1.5B-Instruct
   Epochs: 2
   Estimated time: ~3-4 hours


In [5]:
# Cell 5: Load model WITHOUT quantization (T4 has 15GB, should fit)
print("\n🔄 Loading model...")

from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # Use FP16 instead of 4-bit
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Model loaded: {MODEL_NAME}")
print(f"📊 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB / 15 GB")


🔄 Loading model...


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Model loaded: Qwen/Qwen2.5-1.5B-Instruct
📊 GPU Memory: 2.88 GB / 15 GB


In [6]:
# Cell 6: Configure LoRA (REQUIRED!)
print("\n🔄 Configuring LoRA...")

from peft import LoraConfig, get_peft_model

# Enable gradient checkpointing and disable cache
model.gradient_checkpointing_enable()
model.config.use_cache = False  # CRITICAL: Required for gradient checkpointing

# Prepare model for training
for param in model.parameters():
    param.requires_grad = False  # Freeze base model
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)  # Cast layer norms to fp32

model.enable_input_require_grads()

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ LoRA configured")
print(f"📊 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB / 15 GB")


🔄 Configuring LoRA...
trainable params: 4,358,144 || all params: 1,548,072,448 || trainable%: 0.2815
✅ LoRA configured
📊 GPU Memory: 2.89 GB / 15 GB


In [7]:
# Cell 7: Load dataset
print("\n🔄 Loading dataset...")

from datasets import load_dataset

dataset = load_dataset(
    'json',
    data_files={
        'train': TRAIN_FILE,
        'validation': VAL_FILE
    }
)

print(f"✅ Dataset loaded:")
print(f"   Training: {len(dataset['train'])} examples")
print(f"   Validation: {len(dataset['validation'])} examples")
print(f"\n📋 Sample entry:")
print(dataset['train'][0])


🔄 Loading dataset...


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

✅ Dataset loaded:
   Training: 31409 examples
   Validation: 3717 examples

📋 Sample entry:
{'instruction': 'Analyze this session log chunk and determine if it contains normal or suspicious activity. If suspicious, identify all MITRE ATT&CK techniques and explain why.', 'input': '{"metadata": {"session_id": "20251025_163600", "chunk_index": 220, "start_time": "2025-10-25T10:42:06.341Z", "end_time": "2025-10-25T10:42:06.351Z", "number_of_events": 20}, "logs": [{"timestamp": "2025-10-25T10:42:06.341Z", "event_type": "network", "length": 107, "summary": "Ether / IPv6 / UDP / mDNS Qry b\'cefalo(2)._dosvc._tcp.local.\'", "layers": {"IPv6": {"version": "6", "tc": "0", "fl": "870203", "plen": "53", "nh": "17", "hlim": "1", "src": "fe80::79ec:5385:3926:7cbd", "dst": "ff02::fb"}, "UDP": {"sport": "5353", "dport": "5353"}, "DNS": {"id": "0", "qr": "0", "opcode": "0", "aa": "0", "tc": "0", "rd": "0", "ra": "0", "z": "0", "ad": "0", "cd": "0", "rcode": "0", "qdcount": "1", "ancount": "0", "nscount

In [8]:
# Cell 8: Format and tokenize dataset
print("\n🔄 Formatting and tokenizing dataset...")

# Dataset has columns: instruction, input, output
print(f"Dataset columns: {dataset['train'].column_names}")

def format_prompt(example):
    # Combine instruction + input + output into training format
    return f"""{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""

def tokenize_function(examples):
    # Format each example
    texts = [
        format_prompt({
            'instruction': inst,
            'input': inp,
            'output': out
        })
        for inst, inp, out in zip(
            examples['instruction'],
            examples['input'],
            examples['output']
        )
    ]

    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )

    # Add labels (same as input_ids for causal LM)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

print("\n🔄 Tokenizing...")
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)

print(f"✅ Dataset tokenized")
print(f"   Training: {len(tokenized_dataset['train'])} examples")
print(f"   Validation: {len(tokenized_dataset['validation'])} examples")


🔄 Formatting and tokenizing dataset...
Dataset columns: ['instruction', 'input', 'output']

🔄 Tokenizing...


Map:   0%|          | 0/31409 [00:00<?, ? examples/s]

Map:   0%|          | 0/3717 [00:00<?, ? examples/s]

✅ Dataset tokenized
   Training: 31409 examples
   Validation: 3717 examples


In [9]:
# Cell 9: Set up training arguments (REDUCED for memory)
print("\n🔄 Setting up training arguments...")

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=1,              # Reduced from 2 to 1
    per_device_eval_batch_size=1,               # Reduced from 2 to 1
    gradient_accumulation_steps=8,              # Increased from 4 to 8
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    warmup_steps=100,
    optim="adamw_torch",
    report_to="none",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

print("✅ Training arguments configured")
print(f"   Batch size: 1 (reduced for memory)")
print(f"   Effective batch size: {1 * 8} (still 8 total)")
print(f"   Total steps: ~{len(tokenized_dataset['train']) // (1 * 8) * NUM_EPOCHS}")


🔄 Setting up training arguments...
✅ Training arguments configured
   Batch size: 1 (reduced for memory)
   Effective batch size: 8 (still 8 total)
   Total steps: ~7852


In [10]:
# Cell 10: Create Trainer
print("\n🔄 Creating trainer...")

from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
)

print("✅ Trainer created")
print(f"📊 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB / 15 GB")


🔄 Creating trainer...
✅ Trainer created
📊 GPU Memory: 2.89 GB / 15 GB


In [11]:
# Cell 11: Start training
print("\n🚀 Starting training...\n")

import time
start_time = time.time()

trainer.train()

elapsed_time = time.time() - start_time
print(f"\n✅ Training completed in {elapsed_time/3600:.2f} hours")


🚀 Starting training...



Step,Training Loss,Validation Loss
500,0.212300,0.253941
1000,0.183900,0.232756
1500,0.164600,0.224292
2000,0.153200,0.226155
2500,0.152200,0.224059
3000,0.145500,0.215197
3500,0.148000,0.216673
4000,0.140300,0.220824
4500,0.137700,0.219608
5000,0.136800,0.220055



✅ Training completed in 9.46 hours


In [12]:
# Cell 12: Save the fine-tuned model
print("\n💾 Saving model...")

model.save_pretrained(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print(f"✅ Model saved to: {FINAL_MODEL_DIR}")
print("\n📥 Download model:")
print("   See next cell to create download link")


💾 Saving model...
✅ Model saved to: /kaggle/working/fine_tuned_model

📥 Download model:
   See next cell to create download link


In [13]:
# Cell 13: Download model from Kaggle
print("📦 Creating downloadable zip...")

# Zip the model
!zip -r fine_tuned_model.zip {FINAL_MODEL_DIR}

print(f"✅ Model zipped: fine_tuned_model.zip")
print(f"   Size: {os.path.getsize('fine_tuned_model.zip') / (1024**2):.1f} MB")
print("\n📥 Download: Check the 'Output' section on the right")
print("   Click on fine_tuned_model.zip to download")

📦 Creating downloadable zip...
  adding: kaggle/working/fine_tuned_model/ (stored 0%)
  adding: kaggle/working/fine_tuned_model/tokenizer.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 81%)
  adding: kaggle/working/fine_tuned_model/added_tokens.json (deflated 67%)
  adding: kaggle/working/fine_tuned_model/vocab.json (deflated 61%)
  adding: kaggle/working/fine_tuned_model/special_tokens_map.json (deflated 63%)
  adding: kaggle/working/fine_tuned_model/tokenizer_config.json (deflated 89%)
  adding: kaggle/working/fine_tuned_model/README.md (deflated 65%)
  adding: kaggle/working/fine_tuned_model/chat_template.jinja (deflated 71%)
  adding: kaggle/working/fine_tuned_model/adapter_config.json (deflated 56%)
  adding: kaggle/working/fine_tuned_model/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/fine_tuned_model/merges.txt (deflated 57%)
✅ Model zipped: fine_tuned_model.zip
   Size: 19.2 MB

📥 Download: Check the 'Output' section on the right
   Click on fine_tuned_model.zip to download


# Model Evaluation

Now let's evaluate the fine-tuned model on the test dataset to measure its accuracy.

In [ ]:
# Cell 14: Load the fine-tuned model for evaluation
print("🔄 Loading fine-tuned model for evaluation...")

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Use the model path from training (Kaggle)
# If FINAL_MODEL_DIR doesn't exist, use default Kaggle path
try:
    EVAL_MODEL_PATH = FINAL_MODEL_DIR
except NameError:
    EVAL_MODEL_PATH = "/kaggle/working/fine_tuned_model"

print(f"Loading model from: {EVAL_MODEL_PATH}")

# Load model and tokenizer
eval_model = AutoModelForCausalLM.from_pretrained(
    EVAL_MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(EVAL_MODEL_PATH, trust_remote_code=True)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

eval_model.eval()  # Set to evaluation mode

print(f"✅ Model loaded from: {EVAL_MODEL_PATH}")
print(f"📊 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

: 

In [ ]:
# Cell 15: Load test dataset
print("🔄 Loading test dataset...")

from datasets import load_dataset
import json

# Use DATA_PATH from training configuration (Kaggle)
try:
    TEST_FILE = f"{DATA_PATH}/test.jsonl"
except NameError:
    # Default Kaggle path if DATA_PATH not defined
    TEST_FILE = "/kaggle/input/mitre-datset/test.jsonl"

print(f"Loading test data from: {TEST_FILE}")

test_dataset = load_dataset('json', data_files={'test': TEST_FILE})['test']

print(f"✅ Test dataset loaded: {len(test_dataset)} examples")
print(f"\n📋 Sample test entry:")
print(test_dataset[0])

In [ ]:
# Cell 16: Define evaluation function
print("🔄 Defining evaluation functions...")

def generate_response(model, tokenizer, instruction, input_text, max_new_tokens=256):
    """Generate a response for a given instruction and input."""
    prompt = f"""{instruction}

### Input:
{input_text}

### Response:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode only the generated part (remove the prompt)
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

def calculate_exact_match(pred, target):
    """Calculate exact match accuracy."""
    return 1.0 if pred.strip().lower() == target.strip().lower() else 0.0

def calculate_partial_match(pred, target):
    """Calculate partial match (if prediction contains key parts of target)."""
    pred_lower = pred.strip().lower()
    target_lower = target.strip().lower()
    
    # Check if major keywords from target appear in prediction
    target_words = set(target_lower.split())
    pred_words = set(pred_lower.split())
    
    if len(target_words) == 0:
        return 0.0
    
    overlap = len(target_words.intersection(pred_words))
    return overlap / len(target_words)

print("✅ Evaluation functions defined")

In [ ]:
# Cell 17: Run evaluation on test set
print("🔄 Running evaluation on test set...")
print(f"   This may take a while for {len(test_dataset)} examples...\n")

from tqdm import tqdm
import time

# Limit evaluation to first N examples for speed (optional)
EVAL_LIMIT = 100  # Change to len(test_dataset) for full evaluation
eval_samples = test_dataset.select(range(min(EVAL_LIMIT, len(test_dataset))))

results = []
exact_matches = 0
partial_match_scores = []

start_time = time.time()

for i, example in enumerate(tqdm(eval_samples, desc="Evaluating")):
    # Generate prediction
    prediction = generate_response(
        eval_model,
        eval_tokenizer,
        example['instruction'],
        example['input']
    )
    
    # Calculate metrics
    exact_match = calculate_exact_match(prediction, example['output'])
    partial_match = calculate_partial_match(prediction, example['output'])
    
    exact_matches += exact_match
    partial_match_scores.append(partial_match)
    
    # Store result
    results.append({
        'instruction': example['instruction'],
        'input': example['input'],
        'expected': example['output'],
        'predicted': prediction,
        'exact_match': exact_match,
        'partial_match': partial_match
    })
    
    # Show first few examples
    if i < 3:
        print(f"\n{'='*60}")
        print(f"Example {i+1}:")
        print(f"Input: {example['input'][:100]}...")
        print(f"Expected: {example['output'][:100]}...")
        print(f"Predicted: {prediction[:100]}...")
        print(f"Exact Match: {exact_match}, Partial Match: {partial_match:.2f}")

elapsed = time.time() - start_time

print(f"\n{'='*60}")
print(f"✅ Evaluation completed in {elapsed/60:.2f} minutes")
print(f"\n📊 ACCURACY METRICS (on {len(eval_samples)} samples):")
print(f"   Exact Match Accuracy: {exact_matches / len(eval_samples) * 100:.2f}%")
print(f"   Average Partial Match: {sum(partial_match_scores) / len(partial_match_scores) * 100:.2f}%")
print(f"   Total Samples: {len(eval_samples)}")

In [ ]:
# Cell 18: Detailed analysis of results
print("📊 Detailed Analysis:\n")

# Show distribution of partial match scores
import numpy as np

partial_scores_array = np.array(partial_match_scores)

print(f"Partial Match Score Distribution:")
print(f"   Min: {partial_scores_array.min():.2f}")
print(f"   Max: {partial_scores_array.max():.2f}")
print(f"   Mean: {partial_scores_array.mean():.2f}")
print(f"   Median: {np.median(partial_scores_array):.2f}")
print(f"   Std Dev: {partial_scores_array.std():.2f}")

# Count by score ranges
ranges = [
    (0.0, 0.2, "Very Poor (0-20%)"),
    (0.2, 0.4, "Poor (20-40%)"),
    (0.4, 0.6, "Fair (40-60%)"),
    (0.6, 0.8, "Good (60-80%)"),
    (0.8, 1.0, "Excellent (80-100%)")
]

print(f"\n📈 Score Distribution:")
for low, high, label in ranges:
    count = sum(1 for score in partial_scores_array if low <= score < high)
    percentage = count / len(partial_scores_array) * 100
    print(f"   {label}: {count} ({percentage:.1f}%)")

# Show examples of best and worst predictions
print(f"\n✅ BEST Predictions (highest partial match):")
sorted_results = sorted(results, key=lambda x: x['partial_match'], reverse=True)
for i, result in enumerate(sorted_results[:3]):
    print(f"\n{i+1}. Score: {result['partial_match']:.2f}")
    print(f"   Input: {result['input'][:80]}...")
    print(f"   Expected: {result['expected'][:80]}...")
    print(f"   Predicted: {result['predicted'][:80]}...")

print(f"\n❌ WORST Predictions (lowest partial match):")
for i, result in enumerate(sorted_results[-3:]):
    print(f"\n{i+1}. Score: {result['partial_match']:.2f}")
    print(f"   Input: {result['input'][:80]}...")
    print(f"   Expected: {result['expected'][:80]}...")
    print(f"   Predicted: {result['predicted'][:80]}...")

In [ ]:
# Cell 19: Save evaluation results (optional)
print("💾 Saving evaluation results...")

import json
import pandas as pd

# Save as JSON
results_file = "evaluation_results.json"
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# Save as CSV for easier viewing
df = pd.DataFrame(results)
csv_file = "evaluation_results.csv"
df.to_csv(csv_file, index=False, encoding='utf-8')

print(f"✅ Results saved:")
print(f"   JSON: {results_file}")
print(f"   CSV: {csv_file}")
print(f"\n📊 Summary Statistics:")
print(df[['exact_match', 'partial_match']].describe())